In [1]:
# https://builtin.com/data-science/portfolio-optimization-python

import pyodbc 

import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


In [2]:
funds = [
    'Ashoka India Equity Investment Ord',
    'WisdomTree Physical Gold GBP Daily Hdgd',
    'YFS Argonaut Absolute Return I GBP Acc',
    'iShares Physical Gold ETC GBP',
    'L&G Global 100 Index I Acc',
    'FAST - Global Fund Y-PF-ACC-GBP',
    'iShares Blmbrg Enh Roll Yld Cmd Swap ETF GBP',
    'Ranmore Global Equity Investor GBP',
    'JPM Global Research Enh Idx Eq E Acc',
    'Winton Trend UCITS I GBP Acc',
    'Invesco S&P 500 QVM ETF GBP',
    'WisdomTree Glb Efficient Cor ETFUSDAcc GBP',
    'MFS Meridian Contrarian Value W1 GBP',
    'Orbis OEIC Global Balanced Standard',
    'JupiterMerian Glb Eq AbsRt I £ H Acc',
    # 'Man Dynamic Income I H GBP Cap', - nothing correlated, only started in 2023
    'Prem Miton Tellworth UK Sel Fd - Cl AAcc',
    'Xtrackers MSCI World ex USA 1C GBP'
]

joined_funds = ', '.join([f"'{fund}'" for fund in funds])
print(joined_funds)

'Ashoka India Equity Investment Ord', 'WisdomTree Physical Gold GBP Daily Hdgd', 'YFS Argonaut Absolute Return I GBP Acc', 'iShares Physical Gold ETC GBP', 'L&G Global 100 Index I Acc', 'FAST - Global Fund Y-PF-ACC-GBP', 'iShares Blmbrg Enh Roll Yld Cmd Swap ETF GBP', 'Ranmore Global Equity Investor GBP', 'JPM Global Research Enh Idx Eq E Acc', 'Winton Trend UCITS I GBP Acc', 'Invesco S&P 500 QVM ETF GBP', 'WisdomTree Glb Efficient Cor ETFUSDAcc GBP', 'MFS Meridian Contrarian Value W1 GBP', 'Orbis OEIC Global Balanced Standard', 'JupiterMerian Glb Eq AbsRt I £ H Acc', 'Prem Miton Tellworth UK Sel Fd - Cl AAcc', 'Xtrackers MSCI World ex USA 1C GBP'


In [3]:
# Connect to SQL Server using pyodbc

# https://stackoverflow.com/questions/4657919/sql-server-does-not-exist-or-access-denied
# https://stackoverflow.com/questions/71082494/getting-a-warning-when-using-a-pyodbc-connection-object-with-pandas

from sqlalchemy.engine import URL
connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};server=localhost;database=Portfolio;trusted_connection=yes;'
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

# with engine.connect() as connection:
    # df = pd.read_sql_query('select * from dbo.Instrument', connection)


In [4]:
# get the min and max date for the available adjusted returns 
minMaxDateQuery = f"WITH FundDateRanges as \
  ( \
      select I.Id, I.[Name], min(AR.Date) as minDate, max(AR.Date) as maxDate \
        from dbo.Instrument I \
        join dbo.AdjustedReturn AR on AR.InstrumentId = i.Id \
      where name in \
      ( \
        {joined_funds} \
      ) \
      group by I.Id, I.[Name] \
) \
select max(minDate) as minDate, min(maxDate) as maxDate from FundDateRanges" 
#print(minMaxDateQuery)

with engine.connect() as connection:
    minMaxDate = pd.read_sql_query(minMaxDateQuery, connection)

minDate = minMaxDate['minDate'][0]
maxDate = minMaxDate['maxDate'][0]

#print(f"Min Date: {minDate}, Max Date: {maxDate}")


In [5]:
def get_stock(name):
    query = f"""
    select 
        AR.Date, 
        AR.LogValue as '{name}'
    from 
        dbo.Instrument I
        join dbo.AdjustedReturn AR on AR.InstrumentId = I.Id
    where 
        I.[Name] = '{name}'
        and AR.Date >= '{minDate}'
        and AR.Date <= '{maxDate}'
    order by AR.Date asc
    """
    
    with engine.connect() as connection:
        df = pd.read_sql_query(query, connection)
    
    df.set_index('Date', inplace=True)
    return df

In [6]:
from functools import reduce

def combine_funds(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    print(df_merged.head())
    return df_merged

In [7]:
#gold = get_stock('WisdomTree Physical Gold GBP Daily Hdgd')
#print(gold.head())

portfolio = combine_funds(funds)

#print(portfolio)
#portfolio.to_csv("portfolio.csv", index=False)


            Ashoka India Equity Investment Ord  \
Date                                             
2020-12-02                            0.030772   
2020-12-03                           -0.005698   
2020-12-04                           -0.001907   
2020-12-07                            0.000000   
2020-12-08                            0.007605   

            WisdomTree Physical Gold GBP Daily Hdgd  \
Date                                                  
2020-12-02                                 0.010921   
2020-12-03                                 0.000499   
2020-12-04                                 0.002493   
2020-12-07                                 0.017520   
2020-12-08                                 0.002199   

            YFS Argonaut Absolute Return I GBP Acc  \
Date                                                 
2020-12-02                               -0.005274   
2020-12-03                               -0.008943   
2020-12-04                                0.011

In [8]:
from pypfopt import EfficientFrontier, risk_models, expected_returns

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

In [11]:
# https://builtin.com/data-science/portfolio-optimization-python
mu = mean_historical_return(portfolio, log_returns=True)  
#S = CovarianceShrinkage(portfolio).ledoit_wolf()


C:\Users\Pandelus\AppData\Roaming\Python\Python313\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\Pandelus\AppData\Roaming\Python\Python313\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
C:\Users\Pandelus\AppData\Roaming\Python\Python313\site-packages\pypfopt\expected_returns.py:32: UserWarning: Some returns are NaN. Please check your price data.
  warnings.warn(
C:\Users\Pandelus\AppData\Roaming\Python\Python313\site-packages\pypfopt\expected_returns.py:36: UserWarning: Some returns are infinite. Please check your price data.
  warnings.warn(
